# Matrix Factorization

協調フィルタリングにはアイテム、ユーザーが増えるとデータが膨大になってしまうという欠点があります。Matrix Factorizationは、アイテム、ユーザーの特徴量の次元削減を行うことによって、データの表現を維持しつつ、フィルタリングを行う手法です。
例にすると4人のユーザー、5つのアイテムがあるため、単純に協調フィルタリングを行おうとすると4×5の行列が必要になります。これが数千万単位になるとデータサイズは膨大になります。
そこで、ユーザー、アイテムの特徴量を次元削減し、その積が近似的に元の4×5の行列を表現することを考えます。
今回はこの手法を用いてユーザーに対して良さげな映画をおすすめすることを考えます。

In [53]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader
from typing import Tuple
from collections import defaultdict


In [54]:
class MatrixFactorization(nn.Module):
    def __init__(self, n_users, n_item, k=20):
        super().__init__()
        # kを小さくすればするほど次元削減
        # ユーザーの特徴ベクトル
        self.user_factors = nn.Embedding(n_users, k, sparse=True)
        # アイテムの特徴ベクトル
        self.item_factors = nn.Embedding(n_item, k, sparse=True)

    def forward(self, user, item):
        # 次元削減された特徴ベクトルの内積
        return (self.user_factors(user) * self.item_factors(item)).sum(1)


In [55]:
df = pd.read_csv('/content/ratings.dat', delimiter='::', header=None)
df.columns = ['UserID', 'MovieID', 'Rating', 'Timestamp']
df = df.drop(columns=['Timestamp'])
df.head()


<ipython-input-55-90d45cdc4146>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('/content/ratings.dat', delimiter='::', header=None)


,UserID,MovieID,Rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [56]:
df.shape

(1000209, 3)

In [57]:
rating_matrix = df.pivot(index='UserID', columns='MovieID', values='Rating')
n_users, n_movies = rating_matrix.shape
print(f'num of users: {n_users}  num of items: {n_movies}')
rating_matrix


num of users: 6040  num of items: 3706


MovieID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
UserID,,,,,,,,,,,,,,,,,,,,,
1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,NaN,NaN,NaN,2.0,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


これまでの説明におけるユーザー＝UserIDで識別できる個人、アイテム＝MovieIDで識別できる映画と言えます。ユーザー×アイテムの行列の成分は、ユーザーの映画に対するRatingとなります。ユーザーは全ての映画に対してレビューをしているわけではないので、ほとんどがNaNになっています。
この膨大な行列をMatrix Factorizationにより、次元削減した表現を獲得することが目的です。
この表現を獲得できれば、「あるユーザーのRating（≒好み）は他のユーザーのRatingに似ているから、他のユーザーのRatingが高いこの映画を推薦しよう」というシステムを作ることが可能です。

# Datasetクラスを実装する

In [27]:
class MovieLens1mDataset(Dataset):
    # 定数定義: 各列のインデックス位置
    USER_ID = 0
    MOVIE_ID = 1
    RATING = 2

    def __init__(self, rating_path: str) -> None:
        super().__init__()  # 親クラスのコンストラクタを呼び出す
        # レーティングデータをCSVファイルから読み込む。デリミタは'::'。
        self.df = pd.read_csv(rating_path, delimiter='::', header=None)
        # DataFrameの列名を設定する
        self.df.columns = ['UserID', 'MovieID', 'Rating', 'Timestamp']
        # 'Timestamp'列を削除
        self.df = self.df.drop(columns=['Timestamp'])

    def __getitem__(self, index: int) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        # 指定されたインデックスでDataFrameから行を取得し、その値を取得
        values = self.df.iloc[index].values
        # ユーザーIDと映画IDのインデックスは0から始まるため、1を引く
        user_id = values[self.USER_ID] - 1
        movie_id = values[self.MOVIE_ID] - 1
        # レーティング値を浮動小数点数として取得
        target = np.float32(values[self.RATING])
        # ユーザーID、映画ID、レーティング値をタプルとして返す
        return user_id, movie_id, target

    def __len__(self) -> int:
        # DataFrameの行数を返す
        return len(self.df)


In [28]:
# MovieLens1mDatasetのインスタンスを作成し、指定されたファイルパスからデータをロード
dataset = MovieLens1mDataset('/content/ratings.dat')

# トレーニングデータセットのサイズを全データの70%として計算
n_train = int(len(dataset) * 0.7)
# 検証データセットのサイズを残りの30%として計算
n_val = len(dataset) - n_train
# データセットをトレーニング用と検証用にランダムに分割
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [n_train, n_val])

# バッチサイズを64と定義
BATCH_SIZE = 64

# トレーニング用のデータローダを作成
train_dataloader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,  # バッチサイズを指定
    shuffle=True,           # データをランダムにシャッフル
    pin_memory=True         # データをCPUのピン留めメモリにロード（GPU転送を高速化）
)

# 検証用のデータローダを作成
val_dataloader = DataLoader(
    val_dataset,
    batch_size=1,          # バッチサイズを1として、各ステップで1つのサンプルを処理
    shuffle=True,          # データをランダムにシャッフル
    pin_memory=True        # データをCPUのピン留めメモリにロード（GPU転送を高速化）
)

# データローダーを辞書に格納してアクセスを容易にする
dataloaders = dict(train=train_dataloader, val=val_dataloader)


<ipython-input-27-118a6bbfe60b>:10: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  self.df = pd.read_csv(rating_path, delimiter='::', header=None)


# train_modelを実装する

In [ ]:
from collections import defaultdict
from tqdm import tqdm
import torch

def train_model(model, dataloaders: dict, n_epoch: int, optimizer, criterion):
    # 各フェーズ（訓練、検証）での損失値を追跡するための辞書を初期化
    loss_results = defaultdict(list)

    # 指定されたエポック数で訓練と検証を繰り返す
    for epoch in range(n_epoch):
        # 各エポックにおける訓練と検証の損失を記録する辞書
        loss_per_epoch = dict(train=0, val=0)

        # 'train'と'val'のフェーズをそれぞれ実行
        for phase in ['train', 'val']:
            # モデルを訓練モードまたは検証モードに設定
            if phase == 'train':
                model.train()  # 訓練モード
            else:
                model.eval()   # 検証モード

            # データローダからバッチ単位でデータを取得し処理
            for users, items, targets in tqdm(dataloaders[phase]):
                # 勾配をゼロに初期化（新しいバッチの勾配計算のため）
                optimizer.zero_grad()

                # 勾配の計算を有効化（訓練時のみ）
                with torch.set_grad_enabled(phase == 'train'):
                    preds = model(users, items)  # モデルから予測値を取得
                    loss = criterion(preds, targets)  # 損失を計算
                    loss_per_epoch[phase] += loss.item()  # 損失を累積

                    # 訓練フェーズの場合、バックプロパゲーションとパラメータ更新
                    if phase == 'train':
                        loss.backward()  # 勾配を計算
                        optimizer.step()  # モデルのパラメータを更新

        # 各フェーズの損失をリストに追加
        loss_results['train'].append(loss_per_epoch['train'])
        loss_results['val'].append(loss_per_epoch['val'])

        # エポックごとの損失を表示
        print(f"[epoch {epoch+1}] train loss: {loss_per_epoch['train']}   val loss: {loss_per_epoch['val']}")

    return loss_results  # 訓練と検証の損失を返す


In [31]:
def train_model(model, dataloaders: dict, n_epoch: int, optimizer, criterion):
    # 各フェーズ（訓練、検証）での損失値を追跡するための辞書を初期化
    loss_results = defaultdict(list)

    # 指定されたエポック数で訓練と検証を繰り返す
    for epoch in range(n_epoch):
        # 各エポックにおける訓練と検証の損失を記録する辞書
        loss_per_epoch = dict(train=0, val=0)

        # 'train'と'val'のフェーズをそれぞれ実行
        for phase in ['train', 'val']:
            # モデルを訓練モードまたは検証モードに設定
            if phase == 'train':
                model.train() # 訓練モード
            else:
                model.eval()  # 検証モード

            # データローダからバッチ単位でデータを取得し処理
            for users, items, targets in tqdm(dataloaders[phase]):
                # 勾配をゼロに初期化（新しいバッチの勾配計算のため）
                optimizer.zero_grad()

                # 勾配の計算を有効化（訓練時のみ）
                with torch.set_grad_enabled(phase == 'train'):
                    preds = model(users, items) # モデルから予測値を取得
                    loss = criterion(preds, targets)  # 損失を計算
                    loss_per_epoch[phase] += loss # 損失を累積

                    # 訓練フェーズの場合、バックプロパゲーションとパラメータ更新
                    if phase == 'train':
                        loss.backward() # 勾配を計算
                        optimizer.step()  # モデルのパラメータを更新
        # 各フェーズの損失をリストに追加
        loss_results[phase].append(loss_per_epoch[phase])
        # エポックごとの損失を表示
        print(f"[epoch {epoch+1}] train loss: {loss_per_epoch['train']}   val loss: {loss_per_epoch['val']}")


# 学習する

In [ ]:
# データセットから最大のユーザーIDと映画IDを取得し、それぞれの数を算出（+1して0からカウントするため）
n_users, n_items = dataset.df.UserID.max(), dataset.df.MovieID.max()

# MatrixFactorizationモデルを初期化。ユーザー数、アイテム数、隠れ特徴の次元数kを指定
matrix_factorization = MatrixFactorization(n_users, n_items, k=20)

# 損失関数として平均二乗誤差(Mean Squared Error)を使用
criterion = nn.MSELoss()

# 最適化アルゴリズムとしてSparseAdamを使用し、学習率を0.01に設定
optimizer = optim.SparseAdam(matrix_factorization.parameters(), lr=1e-2)

# 学習を行うエポック数を10と設定
n_epoch = 10

#train_model(matrix_factorization, dataloaders, n_epoch, optimizer, criterion)


In [38]:
# モデルの状態辞書を保存する
#torch.save(matrix_factorization.state_dict(), 'matrix_factorization.pth')

# 保存したモデルの状態辞書をロードする場合
# 新しいモデルインスタンスを作成し、その後保存された状態辞書をロードする
model = MatrixFactorization(n_users, n_items, k=20)
model.load_state_dict(torch.load('matrix_factorization.pth'))
model.eval()  # 推論モードに設定


MatrixFactorization(
  (user_factors): Embedding(6040, 20, sparse=True)
  (item_factors): Embedding(3952, 20, sparse=True)
)

In [61]:
# モデルを評価モードに設定します。これにより、訓練時とは異なり、バッチ正規化やドロップアウトなどの挙動が変化します。
model.eval()

# 検証データセットから次のバッチを取得します。これにはユーザー情報、アイテム情報、および正解のターゲット値が含まれます。
users, items, targets = next(iter(dataloaders['val']))

# モデルを用いて検証データに対する予測を行います。`model(users, items)`により予測が行われ、
# `.detach()`で計算グラフから切り離し、`.numpy()`でNumPy配列に変換します。
# これにより、テンソルが持つ勾配情報を削除し、Pythonの標準的な数値形式に変換します。
print(f'predict: {model(users, items).detach().numpy()} target: {targets.numpy()}')

# 予測結果と実際のターゲット値を出力します。この出力を通じて、モデルの性能を評価することができます。


predict: [5.1865506] target: [5.]


In [48]:
targets.numpy()

array([3.], dtype=float32)